# Generate Drosophila Connectome Weight Matrix

This process builds the synaptic weight matrix used for visual functional profiling (VFP) analysis.

## Output Files

After execution, the output directory will contain:

- **weight_matrix.npz** — sparse matrix in SciPy CSR format  
- **neuron_ids.npy** — list of neuron root IDs  
- **weight_matrix.txt** — text format: `pre_neuron_id post_neuron_id weight_value`

## Parameter Settings

| Parameter | Description | Value |
|-----------|-------------|-------|
| `connections_path` | Path to connection data | `"data/connections.txt"` |
| `synapses_path` | Path to input synapses | `"data/synapses.txt"` |
| `target_pos_sum` | Positive sum for each postsynaptic neuron | 1 |
| `target_neg_sum` | Negative sum for each postsynaptic neuron | -1 |
| `n_jobs` | Number of parallel threads | 20 |
| `results_dir` | Output directory | `"output"` |

## Purpose

The generated weight matrix represents the excitatory and inhibitory connectivity between neurons in the Drosophila connectome and serves as input for downstream VFP analyses.


In [ ]:
from data_process.WeightMatrixExporter import WeightMatrixExporter

exporter = WeightMatrixExporter(
        connections_path="data/connections.txt",
        synapses_path="data/synapses.txt",
        target_pos_sum=1,
        target_neg_sum=-1,
        n_jobs=20,
        results_dir="output"
    )

exporter.save_weight_matrix_npy()

# Compute Visual Functional Profile (VFP)

This process analyzes the Drosophila weight matrix to generate visual functional profiles for neurons.

## Input Files

- **matrix_npz_path** — path to the sparse weight matrix (`VFP.npz`)  
- **neuron_ids_path** — path to the list of neuron IDs (`neuron_ids.npy`)  
- **neuron_types_path** — CSV file mapping neuron IDs to types (`visual_neuron_types.txt`)  

## Parameters

| Parameter | Description | Example Value |
|-----------|-------------|---------------|
| `neuron_types` | Set of neuron types to analyze | `{'L1','L2','L3'}` |
| `side` | Hemisphere to analyze (`left` or `right`) | `'right'` |
| `max_depth` | Maximum recursion depth for connectivity analysis | `100` |
| `min_weight` | Minimum synaptic weight threshold for inclusion | `1e-8` |
| `num_processes` | Number of parallel processes | `20` |
| `blocked_types` | List of neuron types to exclude from analysis | `[]` |

## Purpose

The analyzer computes the Visual Functional Profile for specified neuron types, taking into account excitatory and inhibitory weights, and saves the results for downstream visualization and heatmap generation.


In [ ]:
from data_process.VFPAnalyzer import VFPAnalyzer

analyzer = VFPAnalyzer(
    matrix_npz_path="./output/VFP.npz",
    neuron_ids_path="./output/neuron_ids.npy",
    neuron_types_path="./data/visual_neuron_types.txt"
)

params = {
    'neuron_types': {'L1','L2','L3'},   
    'side': 'right',                      
    'max_depth': 100,                     
    'min_weight': 1e-8,                   
    'num_processes': 20,                  
    'blocked_types': []       
}

analyzer.compute_and_save_VFP(**params)

# Plot Single Neuron Visual Functional Profile (VFP)

This code uses the `VFPShow` class to visualize the connectivity heatmaps of a single neuron across all layers.


In [ ]:
from data_process.VFPShow import VFPShow

vfp_show = VFPShow("./data/column_assignment.txt")

vfp_show.plot_single_neuron_all_layers(neuron_id=720575940608832324,side='100_8', matrices='neuron_matrices')